# Tissue enrichment analysis
The file *Mouse_Gene_Atlas* has been downloaded from [Enrichr](http://amp.pharm.mssm.edu/Enrichr/#stats) on 2018-06-14.  

In [ ]:
library(grDevices)
library(org.Mm.eg.db)
library(ggplot2)
library(Hmisc)

In [ ]:
# Load differentially expressed genes in our study
incSet = unlist(read.csv("RANKL_effect_increased.csv"))
decSet = unlist(read.csv("RANKL_effect_reduced.csv"))

In [ ]:
#NB: identifiers are capitalized outdated mouse gene names
fileMGA = file("Mouse_Gene_Atlas",open="r")
rawMGA = readLines(fileMGA)
close(fileMGA)

In [ ]:
extractTissue <- function(x){
    return(strsplit(x, "\t")[[1]][1])
}
extractGenes <- function(x){
    geneTissue = sapply(strsplit(strsplit(x, "\t")[[1]][-c(1:2)], ","), function(x) x[[1]])
    return(geneTissue)
}

In [ ]:
MGA = lapply(rawMGA, extractGenes)
names(MGA) <- lapply(rawMGA, extractTissue)

In [ ]:
updIncSet = select(org.Mm.eg.db, keys = as.character(incSet), keytype="ALIAS", col="SYMBOL")
updDecSet = select(org.Mm.eg.db, keys = as.character(decSet), keytype="ALIAS", col="SYMBOL")

In [ ]:
length(incSet)
dim(updIncSet)
length(decSet)
dim(updDecSet)
length(which(updIncSet$ALIAS != updIncSet$SYMBOL))
length(which(updDecSet$ALIAS != updDecSet$SYMBOL))

In [ ]:
updIncSet[updIncSet$ALIAS %in% updIncSet$ALIAS[duplicated(updIncSet$ALIAS)],]

In [ ]:
updDecSet[updDecSet$ALIAS %in% updDecSet$ALIAS[duplicated(updDecSet$ALIAS)],]

The symbols used in these lists are up-to-date and/or ambiguous.

In [ ]:
aliasToSymbol = select(org.Mm.eg.db, keys(org.Mm.eg.db, keytype="ALIAS"), keytype="ALIAS", col="SYMBOL", fuzzy=T)
aliasToSymbol$ALIAS = toupper(aliasToSymbol$ALIAS)

In [ ]:
lapply(MGA, function(x) table(x %in% aliasToSymbol$ALIAS))

In [ ]:
MGA$uterus[which(!(MGA$uterus %in% aliasToSymbol$ALIAS))]
MGA$common_myeloid_progenitor[which(!(MGA$common_myeloid_progenitor %in% aliasToSymbol$ALIAS))]

Some of these gene names cannot be converted back to up-to-date symbols.

In [ ]:
updateSymbols <- function(x){
    updSet = aliasToSymbol[aliasToSymbol$ALIAS %in% x,]
    dupliSymbols = updSet$ALIAS[duplicated(updSet$ALIAS)]
    # Return non-ambiguous cases or don't change name it is an ambiguous but uptodate symbol
    return(unique(with(updSet, updSet[(!(ALIAS %in% dupliSymbols))|(ALIAS == toupper(SYMBOL)), 2])))
}

In [ ]:
updMGA = lapply(MGA, updateSymbols)

In [ ]:
n = length(unique(unlist(updMGA),200))
compareTissueToInc <- function(x, set){
    # Get gene names of overexpressed genes in the tissue
    contTab = matrix(c(length(intersect(x, set)), length(setdiff(x, set)), length(setdiff(set, x)), 0), ncol = 2)
    contTab[2,2] = n - sum(contTab)
    ft = fisher.test(contTab, alternative = "greater")
    return(c(pval = ft$p.value, ft$estimate, inter = length(intersect(x, set))))
}

In [ ]:
pdf("enrichment_wi.pdf")
decEnrich = sapply(updMGA, function(x) compareTissueToInc(x, decSet))
decEnrich[1,] = -log10(sapply(decEnrich[1,] * ncol(decEnrich), function(x) min(x, 1)))
rownames(decEnrich)[2] = "oddsratio"
colpal = colorRampPalette(c("#000000", "#FF0000"))(max(decEnrich[3,]))
ggplot(as.data.frame(t(decEnrich)), aes(x=oddsratio, y=pval, color = inter)) +
  scale_colour_gradientn(colours = colpal, limits=c(0,max(decEnrich[3,])),
  breaks=c(0,round(max(decEnrich[3,])/2),max(decEnrich[3,])), name = "Shared genes") + 
  geom_point(size = 5) + theme_light() + theme(text=element_text(size=26, family="sans"), 
    panel.grid.minor=element_blank(), legend.position = c(0.8, 0.6), legend.spacing = unit(10, "cm"),
    legend.background = element_rect(size=0.3, color = "black"), legend.margin=margin(5,5,10,5)) +  
  xlab("Odd-ratio") + ylab("-log10(corrected p-value)") + ggtitle("With arginine") +
  geom_text(aes(label=ifelse(pval>1,gsub("\\_", " ", colnames(decEnrich)),'')), hjust = 1.02, vjust = 1.05, size = 10)
dev.off()

In [ ]:
pdf("enrichment_wo.pdf")
incEnrich = sapply(updMGA, function(x) compareTissueToInc(x, incSet))
incEnrich[1,] = -log10(sapply(incEnrich[1,] * ncol(incEnrich), function(x) min(x, 1)))
rownames(incEnrich)[2] = "oddsratio"
colpal = colorRampPalette(c("#000000", "#FF0000"))(1+max(incEnrich[3,]))
ggplot(as.data.frame(t(incEnrich)), aes(x=oddsratio, y=pval, color = inter)) +
  scale_colour_gradientn(colours = colpal, limits=c(0,max(incEnrich[3,])),
  breaks=c(0,round(max(incEnrich[3,])/2),max(incEnrich[3,])), name = "Shared genes") + 
  geom_point(size = 5) + theme_light() + theme(text=element_text(size=26, family="sans"), 
    panel.grid.minor=element_blank(), legend.position = c(0.8, 0.6), legend.spacing = unit(10, "cm"),
    legend.background = element_rect(size=0.3, color = "black"), legend.margin=margin(5,5,10,5)) +  
  xlab("Odd-ratio") + ylab("-log10(corrected p-value)") + ggtitle("Without arginine") + 
  geom_text(aes(label=ifelse(pval>1,gsub("\\_", " ", colnames(incEnrich)),'')), hjust = 1.02, vjust = 1.05, size = 10)
dev.off()

In [ ]:
decSub = decEnrich[,which(decEnrich[1,] > -log(0.8))]
incSub = incEnrich[,which(incEnrich[1,] > -log(0.8))]
# We want the bars to go the opposite way for the terms decreased in absence of arginine
incSub[2,] = -incSub[2,]
incSub[1,] = -incSub[1,]

# Combine both sets
Sub = cbind(decSub, incSub)
Sub = as.data.frame(t(Sub))

Sub$cdt = capitalize(gsub("\\_", " ", rownames(Sub)))
Sub$cdt = factor(Sub$cdt, levels = Sub$cdt[order(Sub$oddsratio)])

In [ ]:
svg("enrichment_atlas.svg")
ggplot(data=Sub, aes(x=cdt, y=oddsratio, fill = pval)) +
  geom_bar(stat="identity", color = "grey75") + coord_flip() + 
  geom_vline(xintercept = 5.5, linetype = "dashed", color = "grey75", size = 0.5) +
  scale_fill_gradient2(low = "#4A91C4", mid = "#EEEEEE", high = "#F09F4E", name = "-log(FDR)",
    breaks = c(min(Sub$pval),0,max(Sub$pval)), labels = c(abs(round(min(Sub$pval))), 0, round(max(Sub$pval)))) +
  theme_light() + theme(text=element_text(size=20, family="sans"), legend.margin=margin(5,5,10,5),
    legend.background = element_rect(size=0.3, color = "black"), legend.spacing = unit(10, "cm"), 
    legend.position = c(0.75, 0.15), panel.grid.minor=element_blank()) +
  geom_text(aes(label=inter), hjust=c(rep(1,4),rep(0,5)), color="black", size = 8) +
  annotate("text", x = 6.5, y = 16, label= "With\narginine", color = "#F09F4E", size = 8) + 
  annotate("text", x = 4.5, y = 16, label= "Without\narginine", color = "#4A91C4", size = 8) + 
  scale_y_continuous(breaks = c(round(min(Sub$oddsratio)),-10,0,10,20,round(max(Sub$oddsratio))), 
    labels = c(abs(round(min(Sub$oddsratio))),10,0,10,20,abs(round(max(Sub$oddsratio))))) +
  xlab("Overexpression in tissue") + ylab("Odds-ratio") 
dev.off()

In [ ]:
# We want to get the up-genes with arginine that are overexpressed in osteoclasts 
cat(intersect(decSet, updMGA$osteoclasts))

In [ ]:
# We want to get the up-genes without arginine that are overexpressed in macrophage bone marrow 
cat(intersect(incSet, updMGA$macrophage_bone_marrow_0hr))

## Perform gene atlas enrichment for recArg1 differentially expressed genes

In [ ]:
# Load genes up and downregulated by recArg1
load("RNAlists.diff")

In [ ]:
listUpRNASet = select(org.Mm.eg.db, keys = listUpRNA, keytype="ALIAS", col="SYMBOL")
listDownRNASet = select(org.Mm.eg.db, keys = listDownRNA, keytype="ALIAS", col="SYMBOL")

In [ ]:
length(which(listUpRNASet$ALIAS != listUpRNASet$SYMBOL))
length(which(listDownRNASet$ALIAS != listDownRNASet$SYMBOL))

In [ ]:
listDownRNASet[listDownRNASet$ALIAS %in% listDownRNASet$ALIAS[duplicated(listDownRNASet$ALIAS)],]

The symbols used in these lists are up-to-date and/or ambiguous.

In [ ]:
decEnrich = sapply(updMGA, function(x) compareTissueToInc(x, listDownRNA))
decEnrich[1,] = -log10(sapply(decEnrich[1,] * ncol(decEnrich), function(x) min(x, 1)))
rownames(decEnrich)[2] = "oddsratio"
incEnrich = sapply(updMGA, function(x) compareTissueToInc(x, listUpRNA))
incEnrich[1,] = -log10(sapply(incEnrich[1,] * ncol(incEnrich), function(x) min(x, 1)))
rownames(incEnrich)[2] = "oddsratio"

In [ ]:
decSub = decEnrich[,which(decEnrich[1,] > -log(0.8))]
incSub = incEnrich[,which(incEnrich[1,] > -log(0.8))]
# We want the bars to go the opposite way for the terms decreased in absence of arginine
incSub[2,] = -incSub[2,]
incSub[1,] = -incSub[1,]

# Combine both sets
Sub = cbind(decSub, incSub)
Sub = as.data.frame(t(Sub))

Sub$cdt = capitalize(gsub("\\_", " ", rownames(Sub)))
Sub$cdt = factor(Sub$cdt, levels = Sub$cdt[order(Sub$oddsratio)])

In [ ]:
Sub

In [ ]:
svg("enrichment_atlas_recArg.svg")
ggplot(data=Sub, aes(x=cdt, y=oddsratio, fill = pval)) +
  geom_bar(stat="identity", color = "grey75") + coord_flip() + 
  geom_vline(xintercept = 4.5, linetype = "dashed", color = "grey75", size = 0.5) +
  scale_fill_gradient2(low = "#4A91C4", mid = "#EEEEEE", high = "#F09F4E", name = "-log(FDR)", 
    guide = guide_colorbar(direction = "horizontal", title.position = "top"),
    breaks = c(min(Sub$pval),0,max(Sub$pval)), labels = c(abs(round(min(Sub$pval))), 0, round(max(Sub$pval)))) +
  theme_light() + theme(text=element_text(size=20, family="sans"), legend.margin=margin(5,5,10,5),
    legend.background = element_rect(size=0.3, color = "black"), legend.spacing = unit(10, "cm"), 
    legend.position = c(0.78, 0.1), panel.grid.minor=element_blank()) +
  geom_text(aes(label=inter), hjust=c(rep(1,8),rep(0,4)), color="black", size = 8) +
  annotate("text", x = 5.4, y = -2.7, label= "Without\nrecArg1", color = "#F09F4E", size = 8) + 
  annotate("text", x = 3.7, y = 3, label= "With\nrecArg1", color = "#4A91C4", size = 8) + 
  scale_y_continuous(breaks = c(round(min(Sub$oddsratio)),0,round(max(Sub$oddsratio))), 
    labels = c(abs(round(min(Sub$oddsratio))),0,abs(round(max(Sub$oddsratio))))) +
  xlab("Overexpression in tissue") + ylab("Odds-ratio")
dev.off()

In [ ]:
# We want to get the up-genes with arginine that are overexpressed in osteoclasts 
cat(intersect(decSet, updMGA$osteoclasts))

In [ ]:
# We want to get the up-genes without arginine that are overexpressed in macrophage bone marrow 
cat(intersect(incSet, updMGA$macrophage_peri_LPS_thio_0hrs))

In [ ]:
sessionInfo()